In [1]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt

from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
import itertools
from statsmodels.stats.descriptivestats import sign_test
from scipy import stats

# Import data

In [2]:
path_to_data = '/Users/zeleninam2/Documents/projects/cbcl_ksads_work/revision_2023/data'

path_to_ksads_parent = path_to_data+'/abcd-data-release-5.1/core/mental-health/mh_p_ksads_ss.csv'
path_to_ksads_child = path_to_data+'/abcd-data-release-5.1/core/mental-health/mh_y_ksads_ss.csv'
path_to_cbcl = path_to_data+'/abcd-data-release-5.1/core/mental-health/mh_p_cbcl.csv'

In [6]:
ksads_parent = pd.read_csv(path_to_ksads_parent, sep=',', header=0, low_memory=False)
ksads_child = pd.read_csv(path_to_ksads_child, sep=',', header=0, low_memory=False)

In [8]:
ksads_child.head()

,src_subject_id,eventname,ksads_import_id_t,ksads_1_1_t,ksads_1_2_t,ksads_1_3_t,ksads_1_4_t,ksads_1_5_t,ksads_1_6_t,ksads_2_7_t,...,ksads2_13_897_t,ksads2_13_896_t,ksads2_13_895_t,ksads2_13_894_t,ksads2_13_893_t,ksads2_13_892_t,ksads2_13_891_t,ksads2_13_999_t,ksads2_13_1000_t,ksads2_13_429_t
0,NDAR_INV003RTV85,baseline_year_1_arm_1,NDAR_INV003RTV85_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NDAR_INV005V6D2C,baseline_year_1_arm_1,NDAR_INV005V6D2C_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NDAR_INV007W6H7B,baseline_year_1_arm_1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NDAR_INV00BD7VDC,baseline_year_1_arm_1,NDAR_INV00BD7VDC_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NDAR_INV00CY2MDM,baseline_year_1_arm_1,NDAR_INV00CY2MDM_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Load parent data

In [9]:
long_names_parent = ksads_parent.loc[0].to_dict()
ksads_parent = ksads_parent.drop(0)

ksads_parent_bsl = ksads_parent.loc[ksads_parent.eventname == 'baseline_year_1_arm_1'].copy()
ksads_parent_bsl.head()

,src_subject_id,eventname,ksads_import_id_p,ksads_1_843_p,ksads_1_845_p,ksads_1_844_p,ksads_1_840_p,ksads_1_841_p,ksads_1_842_p,ksads_1_847_p,...,ksads2_25_281_p,ksads2_25_823_p,ksads2_25_824_p,ksads2_25_875_p,ksads2_25_876_p,ksads2_26_796_p,ksads2_comments_p,ksads2_duration_p,ksads2_import_id_p,ksads2_timestamp_p
1,NDAR_INV005V6D2C,baseline_year_1_arm_1,NDAR_INV005V6D2C_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NDAR_INV007W6H7B,baseline_year_1_arm_1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NDAR_INV00BD7VDC,baseline_year_1_arm_1,NDAR_INV00BD7VDC_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NDAR_INV00CY2MDM,baseline_year_1_arm_1,NDAR_INV00CY2MDM_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NDAR_INV00HEV6HB,baseline_year_1_arm_1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Load child data

In [10]:
long_names_child = ksads_child.loc[0].to_dict()
ksads_child = ksads_child.drop(0)

ksads_child_bsl = ksads_child.loc[ksads_child.eventname == 'baseline_year_1_arm_1'].copy()
ksads_child_bsl.head()

,src_subject_id,eventname,ksads_import_id_t,ksads_1_1_t,ksads_1_2_t,ksads_1_3_t,ksads_1_4_t,ksads_1_5_t,ksads_1_6_t,ksads_2_7_t,...,ksads2_13_897_t,ksads2_13_896_t,ksads2_13_895_t,ksads2_13_894_t,ksads2_13_893_t,ksads2_13_892_t,ksads2_13_891_t,ksads2_13_999_t,ksads2_13_1000_t,ksads2_13_429_t
1,NDAR_INV005V6D2C,baseline_year_1_arm_1,NDAR_INV005V6D2C_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NDAR_INV007W6H7B,baseline_year_1_arm_1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NDAR_INV00BD7VDC,baseline_year_1_arm_1,NDAR_INV00BD7VDC_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NDAR_INV00CY2MDM,baseline_year_1_arm_1,NDAR_INV00CY2MDM_baselineyear1arm1_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NDAR_INV00HEV6HB,baseline_year_1_arm_1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Leave only depression columns - both parent and child 

In [11]:
depression_column_list_child = [f'ksads_1_{x}_t' for x in [840, 841, 843, 844, 846]]
col_list_child = ['src_subject_id']
col_list_child += depression_column_list_child

depression_column_list_parent = [f'ksads_1_{x}_p' for x in [840, 841, 843, 844, 846]]
col_list_parent = ['src_subject_id']
col_list_parent += depression_column_list_parent

In [12]:
ksads_child_bsl_alldep = ksads_child_bsl.loc[:, col_list_child]
ksads_parent_bsl_alldep = ksads_parent_bsl.loc[:, col_list_parent]

In [13]:
ksads_child_dict = ksads_child_bsl_alldep.to_dict('records')
ksads_parent_dict = ksads_parent_bsl_alldep.to_dict('records')
print(len(ksads_child_dict))
print(len(ksads_parent_dict))

11811
11746


In [14]:
ksads_child_bsl_alldep.head()

,src_subject_id,ksads_1_840_t,ksads_1_841_t,ksads_1_843_t,ksads_1_844_t,ksads_1_846_t
1,NDAR_INV005V6D2C,0.0,0.0,0.0,0.0,0.0
2,NDAR_INV007W6H7B,0.0,0.0,0.0,0.0,0.0
3,NDAR_INV00BD7VDC,0.0,0.0,0.0,0.0,0.0
4,NDAR_INV00CY2MDM,0.0,0.0,0.0,0.0,0.0
5,NDAR_INV00HEV6HB,0.0,0.0,0.0,0.0,0.0


In [15]:
ksads_parent_bsl_alldep.head()

,src_subject_id,ksads_1_840_p,ksads_1_841_p,ksads_1_843_p,ksads_1_844_p,ksads_1_846_p
1,NDAR_INV005V6D2C,0.0,0.0,0.0,0.0,0.0
2,NDAR_INV007W6H7B,0.0,0.0,0.0,0.0,0.0
3,NDAR_INV00BD7VDC,0.0,0.0,0.0,0.0,0.0
4,NDAR_INV00CY2MDM,0.0,0.0,0.0,0.0,0.0
5,NDAR_INV00HEV6HB,0.0,0.0,0.0,0.0,0.0


In [18]:
ksads_child_bsl_alldep.ksads_1_840_t.value_counts()

ksads_1_840_t
0.0      11741
1.0         66
888.0        1
Name: count, dtype: int64

In [20]:
ksads_child_bsl_alldep['depressed_child'] = ((ksads_child_bsl_alldep.ksads_1_840_t == 1.0 ) | \
                                      (ksads_child_bsl_alldep.ksads_1_841_t == 1.0 ) | \
                                      (ksads_child_bsl_alldep.ksads_1_843_t == 1.0 ) | \
                                      (ksads_child_bsl_alldep.ksads_1_844_t == 1.0 ) | \
                                      (ksads_child_bsl_alldep.ksads_1_846_t == 1.0 ))
ksads_child_bsl_alldep['depressed_child'].value_counts()

depressed_child
False    11692
True       119
Name: count, dtype: int64

In [21]:
ksads_parent_bsl_alldep['depressed_parent'] = ((ksads_parent_bsl_alldep.ksads_1_840_p == 1.0 ) | \
                                      (ksads_parent_bsl_alldep.ksads_1_841_p == 1.0 ) | \
                                      (ksads_parent_bsl_alldep.ksads_1_843_p == 1.0 ) | \
                                      (ksads_parent_bsl_alldep.ksads_1_844_p == 1.0 ) | \
                                      (ksads_parent_bsl_alldep.ksads_1_846_p == 1.0 ))
ksads_parent_bsl_alldep['depressed_parent'].value_counts()

depressed_parent
False    11692
True        54
Name: count, dtype: int64

## Merge

In [22]:
ksads_ParentAndChild_bsl = pd.merge(ksads_parent_bsl_alldep, ksads_child_bsl_alldep, on='src_subject_id', indicator=True)

In [23]:
ksads_ParentAndChild_bsl.shape

(11714, 14)

In [24]:
ksads_ParentAndChild_bsl.head()

,src_subject_id,ksads_1_840_p,ksads_1_841_p,ksads_1_843_p,ksads_1_844_p,ksads_1_846_p,depressed_parent,ksads_1_840_t,ksads_1_841_t,ksads_1_843_t,ksads_1_844_t,ksads_1_846_t,depressed_child,_merge
0,NDAR_INV005V6D2C,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,False,both
1,NDAR_INV007W6H7B,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,False,both
2,NDAR_INV00BD7VDC,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,False,both
3,NDAR_INV00CY2MDM,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,False,both
4,NDAR_INV00HEV6HB,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,False,both


In [25]:
ksads_ParentAndChild_bsl = ksads_ParentAndChild_bsl[['src_subject_id', 'depressed_parent', 'depressed_child']]
ksads_ParentAndChild_bsl.head()

,src_subject_id,depressed_parent,depressed_child
0,NDAR_INV005V6D2C,False,False
1,NDAR_INV007W6H7B,False,False
2,NDAR_INV00BD7VDC,False,False
3,NDAR_INV00CY2MDM,False,False
4,NDAR_INV00HEV6HB,False,False


In [26]:
ksads_ParentAndChild_bsl['both_depressed'] = ((ksads_ParentAndChild_bsl.depressed_parent) & (ksads_ParentAndChild_bsl.depressed_child))
ksads_ParentAndChild_bsl['both_depressed'].value_counts()

both_depressed
False    11710
True         4
Name: count, dtype: int64

In [27]:
ksads_ParentAndChild_bsl['both_notdepressed'] = ((ksads_ParentAndChild_bsl.depressed_parent == False) & (ksads_ParentAndChild_bsl.depressed_child == False))
ksads_ParentAndChild_bsl['both_notdepressed'].value_counts()

both_notdepressed
True     11545
False      169
Name: count, dtype: int64

In [28]:
ksads_ParentAndChild_bsl['parentDEP_childNODEP'] = ((ksads_ParentAndChild_bsl.depressed_parent) & (ksads_ParentAndChild_bsl.depressed_child == False))
ksads_ParentAndChild_bsl['parentDEP_childNODEP'].value_counts()

parentDEP_childNODEP
False    11664
True        50
Name: count, dtype: int64

In [29]:
ksads_ParentAndChild_bsl['parentNODEP_childDEP'] = ((ksads_ParentAndChild_bsl.depressed_parent == False) & (ksads_ParentAndChild_bsl.depressed_child))
ksads_ParentAndChild_bsl['parentNODEP_childDEP'].value_counts()

parentNODEP_childDEP
False    11599
True       115
Name: count, dtype: int64

In [30]:
print('Parent and child both report depression')
print(ksads_ParentAndChild_bsl['both_depressed'].value_counts())

print('\nParent and child both dont report depression')
print(ksads_ParentAndChild_bsl['both_notdepressed'].value_counts())

print('\nParent reports depression, child doesnt')
print(ksads_ParentAndChild_bsl['parentDEP_childNODEP'].value_counts())

print('\nChild reports depression, parent doesnt')
print(ksads_ParentAndChild_bsl['parentNODEP_childDEP'].value_counts())

Parent and child both report depression
both_depressed
False    11710
True         4
Name: count, dtype: int64

Parent and child both dont report depression
both_notdepressed
True     11545
False      169
Name: count, dtype: int64

Parent reports depression, child doesnt
parentDEP_childNODEP
False    11664
True        50
Name: count, dtype: int64

Child reports depression, parent doesnt
parentNODEP_childDEP
False    11599
True       115
Name: count, dtype: int64
